<a href="https://colab.research.google.com/github/Luc4P4nt1/Paraphrazer/blob/main/Paraphraser_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Script Colab per Parafrasi PDF (Formule Renderizzate in HTML con MathJax) - CORRETTO v2 (Impostazioni Modello Originali)

# --- 0. Installazione Librerie ---

# Installa le librerie necessarie in ambiente Colab
!pip install -q google-generativeai pymupdf Pillow google-colab markdown-it-py


# --- 1. Importazioni ---

# Importa tutte le librerie richieste per il funzionamento dello script
import google.generativeai as genai
from google.colab import userdata, files, drive
import fitz  # PyMuPDF, usato per estrarre testo e immagini da PDF
import io
import time
import pathlib
import textwrap
import os
import re  # Regular expressions per parsing Markdown e placeholders
import sys
import shutil # Per operazioni su file/cartelle (es. copia immagini)
import traceback # Per stampare stack trace in caso di errori
import math # Per calcoli (es. numero di chunk)
import unicodedata # Per Normalizzazione Unicode
from markdown_it import MarkdownIt # Per convertire Markdown in HTML
from PIL import Image, UnidentifiedImageError # Python Imaging Library per gestire/validare immagini
from html import escape # Per sanitizzare l'HTML se necessario


# --- 2. Configurazione API Key Gemini ---
try:
    GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key Gemini caricata con successo.")
except userdata.SecretNotFoundError:
    print("Errore Critico: Chiave API 'GEMINI_API_KEY' non trovata.")
    print("Assicurati di aver aggiunto la chiave API come 'Secret' in Colab con il nome 'GEMINI_API_KEY'.")
    sys.exit(1)
except Exception as e:
    print(f"Errore Critico Configurazione API: {e}")
    sys.exit(1)


# --- 3. Impostazioni Modello Gemini (RIPRISTINATE COME DA RICHIESTA) ---
MODEL_NAME_REQUESTED = 'gemini-2.5-pro-exp-03-25'
MODEL_FALLBACK_1 = 'gemini-1.5-pro-latest'
MODEL_FALLBACK_2 = 'gemini-1.5-flash' # Flash-latest potrebbe essere più aggiornato

# ATTENZIONE: max_output_tokens: 60000 è molto alto e potrebbe superare
# i limiti effettivi del modello, causando errori o troncamenti imprevisti.
# Il limite standard per 1.5 Pro è 8192 tokens in output.
generation_config = {
    "temperature": 0.6,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 60000, # Valore richiesto, ma potenzialmente problematico
    "response_mime_type": "text/plain"
}

safety_settings = [
  {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
  {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
  {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
  {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

# Logica selezione modello con fallback (RIPRISTINATA COME DA RICHIESTA)
model = None
model_name_in_use = None
model_candidates = [MODEL_NAME_REQUESTED, MODEL_FALLBACK_1, MODEL_FALLBACK_2]

for model_candidate in model_candidates:
    print(f"Tentativo inizializzazione modello: {model_candidate}")
    try:
        # Tenta di inizializzare il modello con la generation_config specificata
        model = genai.GenerativeModel(
            model_name=model_candidate,
            safety_settings=safety_settings,
            generation_config=generation_config # Usa la config richiesta
        )
        model_name_in_use = model_candidate
        print(f"Modello '{model_name_in_use}' inizializzato con successo.")
        break # Esce dal loop se un modello viene inizializzato
    except Exception as e:
        print(f"Errore durante l'inizializzazione del modello '{model_candidate}': {e}")
        # Non stampare "Provo il modello di fallback..." qui, il loop lo fa già

if model is None:
    print("Errore Critico: Nessun modello Gemini disponibile è stato inizializzato.")
    print("Controlla la tua API key, la connessione e la disponibilità dei modelli richiesti (incluso quello sperimentale) nella tua regione.")
    sys.exit(1)


# --- 4. Istruzioni Dettagliate per la Parafrasi (Revisionate Leggermente per Chiarezza) ---
# (Identiche alla versione precedente corretta)
PARAPHRASING_INSTRUCTIONS = """
Devi elaborare il testo fornito seguendo rigorosamente le seguenti istruzioni:

**Parafrasi del testo:**
1.  Riscrivi il testo fornito utilizzando sinonimi, riformulazioni e diverse costruzioni grammaticali, ma **conservando integralmente il significato originale e tutte le informazioni chiave**.
2.  Mantieni **ASSOLUTAMENTE INVARIATI**: nomi propri (persone, aziende, luoghi), acronimi e sigle (es. OIC, IAS, IFRS, CCNL, ROI, EBIT), riferimenti normativi specifici, date, numeri e valori numerici precisi.
3.  Riscrivi le definizioni utilizzando parole diverse ma assicurandoti che il significato tecnico rimanga identico.
4.  Riformula gli elenchi puntati o numerati mantenendo l'ordine, la struttura e il significato di ciascun punto. Parafrasa il testo di ogni punto.
5.  Se una frase è molto lunga e complessa, puoi dividerla in frasi più brevi per chiarezza, ma mantieni la coerenza logica e il flusso del discorso.

**Gestione Placeholders Immagine (MOLTO IMPORTANTE - FORMATO ALFANUMERICO):**
6.  Nel testo di input POTRESTI trovare placeholders nel formato esatto `IMGP<NumPag>xI<IdxImg>` (es. `IMGP5xI2`). **DEVI MANTENERE QUESTI SPECIFICI PLACEHOLDERS ESATTAMENTE COME LI TROVI NEL TESTO DI INPUT E RIPOSIZIONARLI LOGICAMENTE NEL TESTO PARAFRASATO.** La posizione esatta non è garantita, ma cerca di mantenerli nel contesto corretto.
7.  **NON DEVI ASSOLUTAMENTE INVENTARE o AGGIUNGERE ALTRI PLACEHOLDERS `IMGP...xI...` o simili** per immagini che non siano GIA' presenti con quel formato esatto nel testo di INPUT. Ignora qualsiasi altra indicazione di immagine se non è marcata con `IMGP<NumPag>xI<IdxImg>`.

**Gestione Tabelle, Formule, Commenti, Formattazione, Errori:**
8.  **Tabelle (Importante!)**: L'estrazione del testo dal PDF originale potrebbe perdere la struttura della tabella. Se ricevi testo che sembra provenire da una tabella ma senza una chiara struttura, **NON tentare di inventare una tabella Markdown**. Invece, parafrasa i contenuti come testo normale o una lista, cercando di mantenere la relazione logica tra i dati. Se, e solo se, il testo di input contiene GIA' una tabella formattata in Markdown o con una struttura molto chiara basata su testo (es. usando `|` e `-`), allora preserva quella struttura e i dati nelle celle, parafrasando solo eventuali titoli o didascalie esterne.
9.  **Formule/Equazioni/Notazioni Matematiche/Scientifiche (IMPORTANTE):** Vedi punto 14.
10. **Formatta l'output esclusivamente in MARKDOWN**: Usa `## Titoli H2`, `### Titoli H3`, `**grassetto**`, `*corsivo*` (assicurati di lasciare uno spazio prima e dopo la parola/frase in corsivo, es. `testo *parola* testo`, non `testo*parola*testo`), liste con `* ` o `1. `. Mantieni la gerarchia e l'indentazione del testo originale dove possibile. **Nota:** La formattazione originale (come il corsivo) potrebbe essere persa durante l'estrazione dal PDF; applica il corsivo nel testo parafrasato solo se semanticamente appropriato o se indicato esplicitamente con marcatori speciali (non presenti in questo script).
11. **Note a piè di pagina:** Parafrasa il contenuto della nota mantenendo il riferimento originale (es. "[1] Testo parafrasato della nota.").
12. Indica testo illeggibile o mancante con `[TESTO ILLEGGIBILE]`.
13. L'output finale deve essere **UNICAMENTE il testo parafrasato e formattato in Markdown**, contenente SOLO i placeholders `IMGP...xI...`, il codice LaTeX delle formule (es. `$E=mc^2$`) e i marcatori `@@COMMENTO: ... @@`. **NON includere header, footer, o testo introduttivo/conclusivo non richiesto.**

14. **Gestione Specifica Formule (CONFERMATA):** Se identifichi una formula matematica, un'equazione o una notazione scientifica significativa nel testo originale:
    * **NON MODIFICARE ASSOLUTAMENTE la formula originale.** Mantienila esattamente com'è nel testo di input.
    * **ASSICURATI CHE SIA RAPPRESENTATA USANDO LA SINTASSI LATEX STANDARD**, racchiusa tra delimitatori appropriati (`$ ... $` per inline, `$$ ... $$` per display/blocco). Se la formula originale non è già in formato LaTeX, cerca di convertirla in LaTeX standard in modo accurato.
    * **NON usare marcatori speciali come `@@FORMULA:...@@`.** Includi direttamente il codice LaTeX (es. `$E=mc^2$`) nel flusso del testo Markdown dell'output. MathJax nell'HTML finale si occuperà del rendering.
    * **Parafrasa SOLO il testo esplicativo *attorno* alla formula** in modo chiaro e fluente.
    * **Presentazione:** Se una formula è complessa o importante, presentala preferibilmente come blocco display (`$$ ... $$`) su una riga separata nel Markdown.
    * **Esempio Corretto:** Il testo originale "La relazione è E = mc^2." diventa "La relazione fondamentale è espressa come `$E=mc^2$`." nel tuo output Markdown.
    * **NON inventare formule.** Rappresenta solo formule/equazioni/notazioni chiaramente identificabili presenti nel testo originale.

15. **Gestione Specifica Commenti:** Se trovi un marcatore `@@COMMENTO:[spazio][testo del commento qui][spazio]@@` nel testo di input:
    * **Parafrasa il testo del commento** che si trova all'interno del marcatore.
    * **Mantieni il marcatore `@@COMMENTO: ... @@`** attorno al testo del commento parafrasato, assicurandoti che ci sia **esattamente uno spazio** dopo i due punti e **esattamente uno spazio** prima delle doppie chiocciole finali (Formato ESATTO: `@@COMMENTO:[spazio]Testo Parafrasato Qui[spazio]@@`). Questo è cruciale per la successiva conversione in HTML.
    * **Posizionamento:** Mantieni il marcatore del commento nella posizione logica in cui lo trovi nel testo di input.
    * **Esempio:** Se l'input è `Testo originale. @@COMMENTO: Questo è un commento da chiarire. @@`, l'output dovrebbe essere simile a `Testo originale parafrasato. @@COMMENTO: Questa è un'annotazione che necessita di chiarimenti. @@`
"""


# --- 5. Funzioni di Utilità ---
# (Funzioni extract_text_and_images, paraphrase_text_chunk, mount_drive, copy_images_to_drive
#  sono identiche alla versione precedente corretta)

def extract_text_and_images(pdf_content_bytes, temp_image_folder):
    """
    Estrae testo (come testo semplice), immagini (con dimensioni BBox stimate)
    e ANNOTAZIONI TESTUALI da un file PDF.
    NOTA: L'estrazione come testo semplice perde la struttura delle tabelle
    e la formattazione (corsivo, grassetto).
    """
    page_texts = []
    image_references = {} # Dizionario {placeholder: {'path': ..., 'width_cm': ...}}
    doc = None

    try:
        doc = fitz.open(stream=pdf_content_bytes, filetype="pdf")
        n_pages = doc.page_count
        print(f"PDF caricato. Pagine: {n_pages}")

        if not os.path.exists(temp_image_folder):
            os.makedirs(temp_image_folder)
            print(f"Cartella temporanea immagini creata: {temp_image_folder}")

        image_counter_total = 0

        for i in range(n_pages):
            page_num_actual = i + 1
            page = doc.load_page(i)
            # Estrae testo semplice, ordinato. Perde formattazione e struttura tabellare.
            text_content = page.get_text("text", sort=True) or ""

            # Normalizzazione Unicode (importante per consistenza)
            if text_content:
                try:
                    normalized_text = unicodedata.normalize('NFC', text_content)
                    if normalized_text != text_content: print(f"    (Info Pag {page_num_actual}: Testo normalizzato NFC.)")
                    text_content = normalized_text
                except Exception as norm_e: print(f"    ! Attenzione: Errore normalizzazione Unicode Pag {page_num_actual}: {norm_e}")

            # Estrazione Immagini
            page_image_placeholders = []
            try:
                image_list = page.get_images(full=True)
                if image_list:
                    img_count_on_page = 0
                    for img_index, img_info in enumerate(image_list):
                        xref = img_info[0]
                        try:
                            base_image = doc.extract_image(xref)
                            if base_image:
                                img_count_on_page += 1; image_counter_total += 1
                                placeholder = f"IMGP{page_num_actual}xI{img_count_on_page}"
                                image_bytes = base_image["image"]; img_filename = f"{placeholder}.png"
                                local_image_path = os.path.join(temp_image_folder, img_filename)
                                bbox_width_cm = None
                                try:
                                    # Tenta di ottenere il BBox per stimare la larghezza
                                    bbox = page.get_image_bbox(img_info, transform=False)
                                    if bbox.is_valid and not bbox.is_empty:
                                        bbox_width_pt = bbox.width; bbox_width_cm = bbox_width_pt * 2.54 / 72.0
                                except Exception as bbox_e: print(f"      ! Errore get_image_bbox Pag {page_num_actual}, Img {img_count_on_page}: {bbox_e}")

                                try:
                                    # Salva immagine come PNG usando PIL
                                    pil_image = Image.open(io.BytesIO(image_bytes))
                                    if pil_image.mode != 'RGB': pil_image = pil_image.convert('RGB')
                                    pil_image.save(local_image_path, "PNG")
                                    image_references[placeholder] = {'path': local_image_path, 'width_cm': bbox_width_cm}
                                    page_image_placeholders.append(placeholder)
                                except UnidentifiedImageError: print(f"      ! Errore PIL (Formato) Pag {page_num_actual}, Idx {img_index}. Placeholder: {placeholder} IGNORATO.")
                                except Exception as save_e: print(f"      ! Errore Salvataggio PIL Pag {page_num_actual}, Idx {img_index}: {save_e}. Placeholder: {placeholder} IGNORATO.")
                        except Exception as extract_e: print(f"      ! Errore Estrazione Img PyMuPDF Pag {page_num_actual}, Idx {img_index}: {extract_e}. Immagine IGNORATA.")
            except Exception as getimg_e: print(f"    ! Errore recupero elenco immagini Pag {page_num_actual}: {getimg_e}")

            # Estrazione Annotazioni (Commenti)
            page_comments = []
            try:
                annot_count = 0
                for annot in page.annots():
                    # Considera solo annotazioni testuali
                    if annot.type[1] in ['Text', 'FreeText']:
                        info = annot.info; comment_text = info.get("content", "").strip()
                        if comment_text:
                            annot_count += 1
                            try: # Normalizza anche il testo del commento
                                comment_text = unicodedata.normalize('NFC', comment_text)
                            except Exception as norm_e_annot: print(f"    ! Attenzione: Errore normalizzazione annotazione Pag {page_num_actual}: {norm_e_annot}")
                            # Crea il marcatore ESATTO richiesto dalle istruzioni (con spazi)
                            comment_marker = f"@@COMMENTO: {comment_text} @@"
                            page_comments.append(comment_marker)
            except Exception as annot_e: print(f"    ! Errore estrazione annotazioni Pag {page_num_actual}: {annot_e}")

            # Aggiunge placeholder immagini e commenti alla fine del testo della pagina
            # Questo li rende disponibili a Gemini ma separati dal flusso principale.
            # Gemini dovrà poi riposizionarli correttamente nel testo parafrasato.
            if page_image_placeholders: text_content += "\n\n" + "\n".join(page_image_placeholders) + "\n"
            if page_comments: text_content += "\n\n" + "\n".join(page_comments) + "\n"

            page_texts.append(text_content)

        print(f"Estrazione PDF completata. Immagini referenziate: {len(image_references)}")
        return page_texts, image_references

    except fitz.fitz.FileDataError as pdf_err:
        print(f"Errore Fatale Apertura PDF: File non valido o corrotto. {pdf_err}")
        traceback.print_exc(); return None, None
    except Exception as e:
        print(f"Errore Fatale Estrazione PDF: {e}")
        traceback.print_exc(); return None, None
    finally:
        if doc: doc.close(); print("Documento PyMuPDF chiuso.")


def paraphrase_text_chunk(text_chunk, instructions, model, chunk_header):
    """Invia chunk a Gemini per parafrasi. Gestisce errori e retry."""
    if not text_chunk.strip():
        print(f"Chunk '{chunk_header}' vuoto, saltato.")
        return ""

    prompt = f"""{instructions}

--- INIZIO TESTO DA PARAFRASARE ({chunk_header}) ---

{text_chunk}

--- FINE TESTO DA PARAFRASARE ({chunk_header}) ---

Fornisci SOLO il testo parafrasato in MARKDOWN per '{chunk_header}', seguendo TUTTE le istruzioni. Mantieni i placeholder `IMGP...xI...`, le formule LaTeX `$E=mc^2$` / `$$...$$` e i marcatori `@@COMMENTO: Testo Parafrasato Qui @@` (con gli spazi esatti!). Non generare tabelle Markdown se la struttura non è chiarissima nell'input. Assicurati che il testo in corsivo Markdown (*parola*) abbia spazi attorno. **NON INCLUDERE header/footer o altro testo al di fuori del Markdown risultante.**"""

    max_retries = 3 # Tentativi per chiamata API
    base_delay = 5 # Secondi

    for attempt in range(max_retries):
        print(f"  Tentativo {attempt + 1}/{max_retries} invio chunk '{chunk_header}' a Gemini...")
        try:
            response = model.generate_content(prompt)

            # Controllo esplicito se la risposta è stata bloccata
            if not response.candidates:
                 block_reason = "Nessun candidato restituito"
                 safety_ratings_str = "N/A"
                 if response.prompt_feedback:
                     block_reason = str(response.prompt_feedback.block_reason) if response.prompt_feedback.block_reason else block_reason
                     safety_ratings_str = str(response.prompt_feedback.safety_ratings) if response.prompt_feedback.safety_ratings else safety_ratings_str
                 print(f"ATTENZIONE: API ha bloccato la richiesta per {chunk_header} (Tentativo {attempt+1}). Motivo: {block_reason}. Dettagli sicurezza: {safety_ratings_str}")
                 if attempt == max_retries - 1:
                     return f"\n\n[ERRORE_API_BLOCCATA: Chunk '{chunk_header}', Motivo: {block_reason}.]\n\n"
                 delay = base_delay * (attempt + 1)
                 print(f"    Attesa di {delay} secondi prima del prossimo tentativo...")
                 time.sleep(delay)
                 continue

            # Estrazione del testo dalla risposta (primo candidato)
            paraphrased_text = response.text

            # Pulizia base
            cleaned_text = paraphrased_text.strip()

            # Pulizia aggiuntiva per header/footer
            lines = cleaned_text.splitlines()
            lines_filtered = [
                line for line in lines if not (
                    line.strip().startswith(f"--- INIZIO TESTO DA PARAFRASARE ({chunk_header})") or
                    line.strip().startswith(f"--- FINE TESTO DA PARAFRASARE ({chunk_header})") or
                    line.strip() == f"--- {chunk_header} ---" or
                    re.match(r"^\s*markdown\s*$", line, re.IGNORECASE)
                )
            ]
            if lines_filtered and lines_filtered[-1].strip() == "```":
                lines_filtered.pop()

            final_text = "\n".join(lines_filtered).strip()

            if not final_text:
                print(f"Attenzione: Risposta vuota da Gemini per {chunk_header} dopo pulizia.")
                return f"\n\n[RISPOSTA_VUOTA: Chunk '{chunk_header}'.]\n\n"

            print(f"  Chunk '{chunk_header}' parafrasato con successo (Tentativo {attempt+1}).")
            return final_text

        except Exception as e:
            print(f"ERRORE API Gemini o elaborazione risposta per {chunk_header} (Tentativo {attempt+1}): {e}")
            if not isinstance(e, (genai.types.StopCandidateException, genai.types.BlockedPromptException)):
                 traceback.print_exc()

            if attempt == max_retries - 1:
                return f"\n\n[ERRORE_API_GENERAZIONE: Chunk '{chunk_header}'. {e}]\n\n"
            delay = base_delay * (attempt + 1)
            print(f"    Attesa di {delay} secondi prima del prossimo tentativo...")
            time.sleep(delay)

    return f"\n\n[ERRORE_API_GENERAZIONE: Chunk '{chunk_header}'. Falliti tutti i {max_retries} tentativi.]\n\n"


def mount_drive():
    """Monta Google Drive."""
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive montato con successo in /content/drive.")
        return True
    except Exception as e:
        print(f"Errore durante il montaggio di Google Drive: {e}")
        return False

def copy_images_to_drive(local_folder, drive_folder_path):
    """Copia immagini (PNG estratte) da cartella locale a Drive. Crea cartella Drive se non esiste."""
    if not os.path.exists(local_folder):
        print(f"Informazione: Cartella locale sorgente '{local_folder}' non trovata (probabilmente nessuna immagine estratta). Salto copia.")
        return True

    try:
        os.makedirs(drive_folder_path, exist_ok=True)
        print(f"Cartella Drive di destinazione '{drive_folder_path}' verificata/creata.")
    except OSError as e:
        print(f"Errore creazione cartella Drive '{drive_folder_path}': {e}")
        return False

    print(f"Avvio copia immagini da '{local_folder}' a '{drive_folder_path}'...")
    copied_count = 0
    error_count = 0
    files_in_source = os.listdir(local_folder)

    if not files_in_source:
        print("Nessun file immagine trovato nella cartella locale. Copia completata.")
        return True

    for filename in files_in_source:
        if filename.lower().endswith(".png"):
            source_path = os.path.join(local_folder, filename)
            destination_path = os.path.join(drive_folder_path, filename)
            if os.path.isfile(source_path):
                try:
                    shutil.copy2(source_path, destination_path)
                    copied_count += 1
                except Exception as e:
                    print(f"    ! Errore durante la copia del file '{filename}': {e}")
                    error_count += 1

    print(f"Copia immagini completata. File copiati: {copied_count}. Errori: {error_count}.")
    return error_count == 0


# --- Funzione Chiave per Creazione Documento HTML (CON FIX PER COMMENTI) ---
# (Identica alla versione precedente corretta)
def create_html_doc_with_images_and_mathjax(paraphrased_markdown, image_references, output_html_path, drive_image_folder_relative_path):
    """
    Crea un file HTML:
    - Converte Markdown in HTML usando markdown-it-py.
    - Sostituisce i placeholder immagine con tag <img>.
    - Sostituisce i marcatori @@COMMENTO:...@@ con HTML stilizzato (CON FIX OVERFLOW).
    - Include script MathJax per rendering LaTeX.
    """
    print(f"\n--- Inizio Creazione Documento HTML ---")
    print(f"File HTML di output: {output_html_path}")
    print(f"Percorso relativo immagini nell'HTML: {drive_image_folder_relative_path}")

    # Configura MathJax (versione 3) per rendering formule LaTeX ($...$ e $$...$$)
    mathjax_script = """
<script>
MathJax = {
  tex: {
    inlineMath: [['$', '$']],
    displayMath: [['$$', '$$']],
    processEscapes: true,
    processEnvironments: true
  },
  options: {
    skipHtmlTags: ['script', 'noscript', 'style', 'textarea', 'pre', 'code'],
    processHtmlClass: 'markdown-body'  // NUOVA OPZIONE IMPORTANTE
  },
  startup: {
    typeset: true
  }
};
</script>
<script src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js" id="MathJax-script"></script>

"""

    # Inizializza il parser Markdown
    md = MarkdownIt()

    # 1. Pre-processa il Markdown PRIMA della conversione HTML
    processed_markdown = paraphrased_markdown

    # Sostituisci i placeholder IMGP... con tag HTML <img>
    print("  Sostituzione placeholder immagini con tag <img>...")
    placeholders_found = 0
    for placeholder, img_data in image_references.items():
        if isinstance(img_data, dict) and 'path' in img_data:
            img_filename = os.path.basename(img_data['path'])
            relative_img_src = f"{drive_image_folder_relative_path.strip('/')}/{img_filename}"
            safe_relative_img_src = escape(relative_img_src)
            img_tag = f'<img src="{safe_relative_img_src}" alt="{escape(placeholder)}" style="max-width: 90%; height: auto; display: block; margin-left: auto; margin-right: auto; border: 1px solid #eee; padding: 5px;">'
            count = 0
            processed_markdown, count = re.subn(re.escape(placeholder), img_tag, processed_markdown)
            if count > 0:
                placeholders_found += count
        else:
            print(f"    ! Attenzione: Riferimento immagine non valido per '{placeholder}', placeholder lasciato nel testo.")
    print(f"  Sostituiti {placeholders_found} placeholder immagine.")


    # Sostituisci i marcatori @@COMMENTO: ... @@ con HTML stilizzato (CON FIX OVERFLOW)
    print("  Sostituzione marcatori commento con HTML stilizzato...")
    comment_pattern = re.compile(r"@@COMMENTO:\s(.*?)\s@@") # Richiede spazi esatti
    # Stile CSS corretto per gestire l'overflow del testo
    comment_html_template = r'<p style="color: #006400; font-style: italic; border: 1px solid #228B22; background-color: #f0fff0; padding: 10px; margin-top: 15px; margin-bottom: 15px; border-radius: 5px; overflow-wrap: break-word; word-wrap: break-word;"><strong>[Commento:]</strong> \1</p>'

    comments_found = 0
    processed_markdown_temp = processed_markdown
    matches = list(comment_pattern.finditer(processed_markdown_temp))
    comments_found = len(matches)

    processed_markdown = comment_pattern.sub(comment_html_template, processed_markdown_temp)
    print(f"  Sostituiti {comments_found} marcatori commento.")
    if comments_found == 0:
         print("  (Nessun marcatore commento nel formato '@@COMMENTO: ... @@' trovato per la sostituzione)")

    # 2. Converti il Markdown (con tag img/commenti già inseriti) in HTML
    print("  Conversione Markdown -> HTML con markdown-it-py...")
    try:
        # Per debuggare tabelle/corsivo:
        # print("\n--- DEBUG: MARKDOWN PRIMA DI RENDER ---\n", processed_markdown[:2000], "\n------\n")
        html_content = md.render(processed_markdown).replace('&#36;', '$')
        print("  Conversione Markdown -> HTML completata.")
    except Exception as md_err:
        print(f"!!! ERRORE CRITICO durante la conversione Markdown->HTML: {md_err}")
        print("Salvataggio del Markdown grezzo con HTML di base come fallback.")
        traceback.print_exc()
        html_content = f'<p style="color:red; font-weight:bold;">Errore nella conversione Markdown, visualizzazione grezza del Markdown processato:</p>\n<pre style="white-space: pre-wrap; word-wrap: break-word; border: 1px solid red; padding: 10px;">{escape(processed_markdown)}</pre>'

    # 3. Costruisci il documento HTML completo con CSS migliorato
    print("  Costruzione documento HTML finale...")
    # (HTML e CSS identici alla versione precedente corretta)
    full_html = f"""<!DOCTYPE html>
<html lang="it">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Documento Parafrasato</title>
    {mathjax_script}
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            line-height: 1.7;
            padding: 25px;
            max-width: 960px;
            margin: 20px auto;
            background-color: #fdfdfd;
            color: #333;
            font-size: 16px;
        }}
        h1, h2, h3, h4, h5, h6 {{
            color: #2c3e50;
            margin-top: 1.5em;
            margin-bottom: 0.8em;
            line-height: 1.3;
            font-weight: 600;
        }}
        h1 {{ font-size: 2.2em; border-bottom: 2px solid #e0e0e0; padding-bottom: 0.3em;}}
        h2 {{ font-size: 1.8em; border-bottom: 1px solid #eee; padding-bottom: 0.2em;}}
        h3 {{ font-size: 1.4em; }}
        p {{ margin-bottom: 1.2em; }}
        a {{ color: #3498db; text-decoration: none; }}
        a:hover {{ text-decoration: underline; color: #2980b9; }}
        code {{
             background-color: #eef1f3;
             padding: 3px 6px;
             border-radius: 4px;
             font-family: 'Consolas', 'Monaco', 'Courier New', monospace;
             font-size: 0.95em;
             color: #333;
        }}
        pre {{
            background-color: #eef1f3;
            padding: 15px;
            border-radius: 5px;
            overflow-x: auto;
            margin-bottom: 1.5em;
        }}
        pre > code {{
             display: block;
             padding: 0;
             background-color: transparent;
             border-radius: 0;
        }}
        table {{
            border-collapse: collapse;
            width: 100%;
            margin-bottom: 1.5em;
            border: 1px solid #dfe6e9;
            font-size: 0.98em;
        }}
        th, td {{
            border: 1px solid #dfe6e9;
            padding: 10px 12px;
            text-align: left;
            vertical-align: top;
        }}
        th {{
            background-color: #f6f8fa;
            font-weight: 600;
            color: #2c3e50;
        }}
        tbody tr:nth-child(odd) {{
             background-color: #fcfcfc;
        }}
        img {{
             max-width: 100%;
             height: auto;
             display: block;
             margin: 1.5em auto;
             border: 1px solid #e0e0e0;
             border-radius: 4px;
             padding: 5px;
        }}
        ul, ol {{ padding-left: 30px; margin-bottom: 1em; }}
        li {{ margin-bottom: 0.5em; }}
        hr {{ border: none; border-top: 1px solid #eee; margin: 2em 0; }}

    </style>
</head>
<body>
    <h1>Documento Parafrasato</h1>
    <hr>
    <div class="markdown-body">
        {html_content}
    </div>
</body>
</html>"""

    # 4. Salva il file HTML
    print("  Salvataggio file HTML...")
    try:
        with open(output_html_path, 'w', encoding='utf-8') as f:
            f.write(full_html)
        print(f"--- Documento HTML salvato con successo ---")
        return True
    except Exception as e:
        print(f"!!! ERRORE CRITICO salvataggio HTML: {e}")
        traceback.print_exc()
        return False


# --- 7. Funzione Principale di Esecuzione ---
# (Identica alla versione precedente corretta)
def run_full_process():
    """
    Flusso completo: setup, Drive, PDF upload, estrazione (testo semplice, img+size, annotazioni),
    parafrasi (con gestione errori), copia img, creazione HTML con MathJax (con fix commenti).
    LIMITAZIONI NOTE: Tabelle e formattazione (corsivo) probabilmente perse/non renderizzate correttamente.
    """
    print("=============================================")
    print("--- Avvio Processo Parafrasi PDF v7 (Output HTML con Formule Renderizzate) ---")
    print("=============================================")
    # Impostazioni
    CHUNK_SIZE_PAGES = 15 # Numero di pagine PDF per ogni chiamata a Gemini
    TEMP_IMAGE_FOLDER = "/content/temp_pdf_parafrasi_images" # Cartella temporanea locale per immagini
    DRIVE_BASE_OUTPUT_FOLDER = "/content/drive/MyDrive/PDF_Parafrasi_Output_HTML" # Cartella base output su Drive

    # Pulisci cartella temporanea immagini precedente, se esiste
    if os.path.exists(TEMP_IMAGE_FOLDER):
        print(f"Pulizia cartella temporanea immagini precedente: {TEMP_IMAGE_FOLDER}")
        try:
            shutil.rmtree(TEMP_IMAGE_FOLDER)
        except Exception as e:
            print(f"Attenzione: Errore durante la rimozione della cartella temporanea '{TEMP_IMAGE_FOLDER}': {e}")

    # 1. Montaggio Google Drive
    print("\n--- 1. Montaggio Google Drive ---")
    if not mount_drive():
        print("ERRORE CRITICO: Montaggio Google Drive fallito. Impossibile continuare.")
        return

    # 2. Setup Cartelle di Output su Drive
    print("\n--- 2. Configurazione Output su Drive ---")
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    drive_output_subfolder = os.path.join(DRIVE_BASE_OUTPUT_FOLDER, f"Output_{timestamp}")
    drive_image_folder_relative = "Images"
    drive_image_folder_absolute = os.path.join(drive_output_subfolder, drive_image_folder_relative)
    output_html_filename = f"output_parafrasato_{timestamp}.html"
    output_html_path = os.path.join(drive_output_subfolder, output_html_filename)

    print(f"Output HTML verrà salvato in: {output_html_path}")
    print(f"Le immagini verranno salvate in: {drive_image_folder_absolute}")

    try:
        os.makedirs(drive_image_folder_absolute, exist_ok=True)
        print("Cartelle Drive per output create/verificate con successo.")
    except OSError as e:
        print(f"ERRORE CRITICO Creazione Cartelle Output su Drive: {e}")
        return

    # 3. Caricamento File PDF
    print("\n--- 3. Caricamento File PDF ---")
    print("Seleziona e carica il file PDF dal tuo computer...")
    uploaded_pdf_content = None
    uploaded_file_name = None
    try:
        uploaded = files.upload()
        if not uploaded:
            print("Nessun file caricato. Interruzione.")
            return
        uploaded_file_name = list(uploaded.keys())[0]
        uploaded_pdf_content = uploaded[uploaded_file_name]
        print(f"File '{uploaded_file_name}' caricato ({len(uploaded_pdf_content)} bytes).")
    except Exception as e:
        print(f"ERRORE CRITICO durante il Caricamento del PDF: {e}")
        return

    # 4. Estrazione Contenuto PDF
    print("\n--- 4. Estrazione Contenuto PDF (Testo, Immagini, Annotazioni) ---")
    page_texts_with_placeholders, image_references = extract_text_and_images(
        uploaded_pdf_content, TEMP_IMAGE_FOLDER
    )

    if page_texts_with_placeholders is None or image_references is None:
        print("ERRORE CRITICO durante l'estrazione dei contenuti dal PDF. Interruzione.")
        return
    print(f"Numero pagine elaborate: {len(page_texts_with_placeholders)}")
    if image_references:
        print(f"Numero immagini referenziate: {len(image_references)}")
    else:
        print("Nessuna immagine trovata o estratta dal PDF.")

    # 5. Parafrasi con Gemini (Chunking)
    print("\n--- 5. Inizio Parafrasi con Gemini ---")
    full_paraphrased_markdown = ""
    total_pages = len(page_texts_with_placeholders)
    total_chunks = math.ceil(total_pages / CHUNK_SIZE_PAGES) if total_pages > 0 else 0
    print(f"Divisione del testo in {total_chunks} chunk di max {CHUNK_SIZE_PAGES} pagine ciascuno.")
    print(f"Modello Gemini in uso: {model_name_in_use} (Config: {generation_config})") # Log config

    if total_chunks == 0:
        print("Il PDF non contiene testo estraibile. Impossibile parafrasare.")
    else:
        for i in range(0, total_pages, CHUNK_SIZE_PAGES):
            chunk_num = (i // CHUNK_SIZE_PAGES) + 1
            start_page_num = i + 1
            end_page_index = min(i + CHUNK_SIZE_PAGES, total_pages)
            end_page_num = end_page_index
            chunk_header = f"PAGINE {start_page_num}-{end_page_num}"

            print(f"\nProcessando Chunk {chunk_num}/{total_chunks} ({chunk_header})...")

            text_chunk_to_process = "\n\n".join(page_texts_with_placeholders[i:end_page_index])

            paraphrased_chunk_md = paraphrase_text_chunk(
                text_chunk_to_process,
                PARAPHRASING_INSTRUCTIONS,
                model,
                chunk_header
            )

            if paraphrased_chunk_md and not paraphrased_chunk_md.startswith("[ERRORE") and not paraphrased_chunk_md.startswith("[RISPOSTA_VUOTA"):
                full_paraphrased_markdown += paraphrased_chunk_md.strip() + "\n\n"
            elif paraphrased_chunk_md.startswith("[ERRORE"):
                print(f"!!! Errore durante la parafrasi del chunk {chunk_header}. Dettagli sopra. Aggiungo marcatore errore al documento finale.")
                full_paraphrased_markdown += f"\n\n**[ERRORE NELLA PARAFRASI DEL CHUNK: {chunk_header}]**\n{paraphrased_chunk_md}\n\n"
            else: # Risposta vuota o saltata
                print(f"Attenzione: Chunk {chunk_header} ha prodotto una risposta vuota o è stato saltato. Aggiungo marcatore.")
                full_paraphrased_markdown += f"\n\n**[CHUNK VUOTO O SALTATO: {chunk_header}]**\n{paraphrased_chunk_md}\n\n"

            pause_duration = max(5, CHUNK_SIZE_PAGES // 3) # Pausa tra chunk
            print(f"  Pausa di {pause_duration} secondi prima del prossimo chunk...")
            time.sleep(pause_duration)

    print("\n--- Parafrasi di tutti i chunk completata ---")

    print("\n--- Controllo Output Markdown Generato (Primi/Ultimi 500 caratteri) ---")
    print("Inizio:")
    print(textwrap.shorten(full_paraphrased_markdown, width=500, placeholder="..."))
    print("\nFine:")
    print("..." + full_paraphrased_markdown[-500:])
    print("-----------------------------------------------------\n")


    # 6. Copia Immagini su Drive
    if image_references:
        print("\n--- 6. Copia Immagini Estraette su Drive ---")
        copy_success = copy_images_to_drive(TEMP_IMAGE_FOLDER, drive_image_folder_absolute)
        if not copy_success:
            print("ATTENZIONE: Si sono verificati errori durante la copia delle immagini su Drive.")
        else:
            print("Copia delle immagini su Drive completata.")
    else:
        print("\n--- 6. Copia Immagini Estraette su Drive ---")
        print("Nessuna immagine estratta, quindi nessuna immagine da copiare.")

    # 7. Creazione Documento HTML Finale
    print("\n--- 7. Creazione Documento HTML Finale con MathJax ---")
    html_creation_success = create_html_doc_with_images_and_mathjax(
        full_paraphrased_markdown,
        image_references,
        output_html_path,
        drive_image_folder_relative
    )

    # 8. Pulizia Cartella Temporanea Locale
    print("\n--- 8. Pulizia ---")
    if os.path.exists(TEMP_IMAGE_FOLDER):
        print(f"Rimozione cartella temporanea immagini locale: {TEMP_IMAGE_FOLDER}")
        try:
            shutil.rmtree(TEMP_IMAGE_FOLDER)
        except Exception as e:
            print(f"Attenzione: Errore durante la rimozione della cartella temporanea '{TEMP_IMAGE_FOLDER}': {e}")

    # Messaggio finale
    print("\n=============================================")
    if html_creation_success:
        print(f"--- PROCESSO COMPLETATO CON SUCCESSO ---")
        print(f"File HTML generato: {output_html_path}")
        print(f"Immagini (se presenti) salvate in: {drive_image_folder_absolute}")
        print("\nPuoi trovare i file nella tua cartella Google Drive.")
        print("Apri il file .html nel tuo browser per vedere il documento parafrasato con le formule renderizzate.")
    else:
        print(f"--- PROCESSO TERMINATO CON ERRORI ---")
        print("Errore durante la creazione del file HTML finale.")
        print("Controllare i messaggi di log sopra per i dettagli.")
        print(f"Potrebbero comunque essere stati creati output parziali (testo, immagini) nella cartella Drive: {drive_output_subfolder}")
    print("=============================================")

# --- 8. Esecuzione ---
run_full_process()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.5 MB/s eta 0:00:00
API Key Gemini caricata con successo.
Tentativo inizializzazione modello: gemini-2.5-pro-exp-03-25
Modello 'gemini-2.5-pro-exp-03-25' inizializzato con successo.
--- Avvio Processo Parafrasi PDF v7 (Output HTML con Formule Renderizzate) ---

--- 1. Montaggio Google Drive ---
Mounted at /content/drive
Google Drive montato con successo in /content/drive.

--- 2. Configurazione Output su Drive ---
Output HTML verrà salvato in: /content/drive/MyDrive/PDF_Parafrasi_Output_HTML/Output_20250411_094330/output_parafrasato_20250411_094330.html
Le immagini verranno salvate in: /content/drive/MyDrive/PDF_Parafras

Saving Economia e gestione della banca - Migliavacca.pdf to Economia e gestione della banca - Migliavacca.pdf
File 'Economia e gestione della banca - Migliavacca.pdf' caricato (13599421 bytes).

--- 4. Estrazione Contenuto PDF (Testo, Immagini, Annotazioni) ---
PDF caricato. Pagine: 212
Cartella temporanea immagini creata: /content/temp_pdf_parafrasi_images
Estrazione PDF completata. Immagini referenziate: 195
Documento PyMuPDF chiuso.
Numero pagine elaborate: 212
Numero immagini referenziate: 195

--- 5. Inizio Parafrasi con Gemini ---
Divisione del testo in 15 chunk di max 15 pagine ciascuno.
Modello Gemini in uso: gemini-2.5-pro-exp-03-25 (Config: {'temperature': 0.6, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 60000, 'response_mime_type': 'text/plain'})

Processando Chunk 1/15 (PAGINE 1-15)...
  Tentativo 1/3 invio chunk 'PAGINE 1-15' a Gemini...
  Chunk 'PAGINE 1-15' parafrasato con successo (Tentativo 1).
  Pausa di 5 secondi prima del prossimo chunk...

Processando Chunk 2/